In [1]:
import numpy as np     #只需要下载numpy库即可
import random
import GridWorld_v2
import time
from IPython.display import clear_output

In [2]:
rows = 5      #记得行数和列数这里要同步改
columns = 5

# gridworld = GridWorld_v2.GridWorld_v2(rows=rows, columns=columns, forbiddenAreaNums=8, targetNums=2, seed = 52,forbiddenAreaScore=-10)
# gridworld = GridWorld_v2.GridWorld_v2(desc = [".#",".T"])             #赵老师4-1的例子
# gridworld = GridWorld_v2.GridWorld_v2(desc = ["##.T","...#","...."])  #随便弄的例子
gridworld = GridWorld_v2.GridWorld_v2(forbiddenAreaScore=-10, score=1,desc = [".....",".##..","..#..",".#T#.",".#..."]) 
#gridworld = GridWorld_v2(forbiddenAreaScore=-10, score=1,desc = ["T."]) 
gridworld.show()


value = np.zeros(rows*columns)       #初始化可以任意，也可以全0
qtable = np.zeros((rows*columns,5))  #初始化，这里主要是初始化维数，里面的内容会被覆盖所以无所谓


⬜️⬜️⬜️⬜️⬜️
⬜️🚫🚫⬜️⬜️
⬜️⬜️🚫⬜️⬜️
⬜️🚫✅🚫⬜️
⬜️🚫⬜️⬜️⬜️


In [93]:
def SARSA(gridworld:GridWorld_v2.GridWorld_v2, grad, w, gamma = 0.99,trajectorySteps=-1, learning_rate=0.001, final_epsilon=0.01, num_episodes=600)->GridWorld_v2.GridWorld_v2:
    """
    这是最基础的SARSA算法

    Parameters:
    trajectorySteps (int): 寻路的轨迹长度，如果是-1，则为寻到目的则停止，否则参数即为trajectory长度
    learning_rate (float): 学习率，用于调节TD-target
    epsilon (float): epsilon-greedy的核心参数，0~1的浮点数，其中1则表示当前state所有决策概率一样，0则表示决策没有任何的随机性
    num_episodes (int): 表示模型迭代次数

    Returns:
    GridWorld_v2.GridWorld_v2: 把模型返回回去
    """
    
    state_value = np.zeros((rows * columns))
    # action_value = np.zeros((rows * columns, 5))
    policy = np.eye(5)[np.random.randint(0,5,size=(rows*columns))] 
    epsilon = 0.5


    if(epsilon > final_epsilon) :
        epsilon -= 0.001
    else:
        epsilon = final_epsilon

    # p1是目标方向的概率，p0是另外四个方向的概率
    p1 = 1-epsilon * (4/5)
    p0 = epsilon/5
    d = {1:p1, 0:p0}
    # policy_epsilon是policy取epsilon-greedy的概率决策
    print("p1",p1,"p0",p0)
    policy_epsilon = np.vectorize(d.get)(policy)

    if trajectorySteps==-1:
        stop_when_reach_target = True
    Trajectorys_cnt = 50
    
    initState=10
    Trajectorys = [gridworld.getTrajectoryScore(nowState=initState, 
                                              action=random.randint(0,4), 
                                              policy=policy_epsilon, 
                                              steps=trajectorySteps, 
                                              stop_when_reach_target=True) for i in range(Trajectorys_cnt)]
    
    for episode in range(num_episodes):

        

        #cnt数组用来检查每个state有多少次访问
        cnt = [0 for i in range(25)]
        
        initState=10
        initAction=random.randint(0,4)  
        
        for Trajectory in Trajectorys:
            steps = len(Trajectory) - 1
            if steps > 10000:
                continue
            for k in range(steps,-1,-1):
                #State，Action，Reward，NextState，NextAction
                tmpstate, tmpaction, tmpscore, nextState, nextAction  = Trajectory[k]
                cnt[tmpstate] += 1
                #SARSA
                nowX, nowY, nextX, nextY = tmpstate//5,tmpstate%5,nextState//5,nextState%5
                # TD_error = action_value[tmpstate][tmpaction] - (tmpscore + gamma * action_value[nextState][nextAction])
                # action_value[tmpstate][tmpaction] -= learning_rate * TD_error
                TD_error = (tmpscore + gamma * grad(nextX,nextY,nextAction) @ w) - grad(nowX, nowY,tmpaction) @ w
                w += learning_rate * TD_error * grad(nowX, nowY,tmpaction)
            # print(TD_error)
            # print(w)
            # print(grad(tmpstate,tmpaction))
            # print((TD_error * grad(tmpstate,tmpaction)).mean())
        # policy improvement
        tmp_policy = []
        for i in range(25):
            x = i//5
            y = i%5
            maxx = grad(x,y,0)@w
            index = 0
            for action in range(5):
                # print(grad(x,y,action)@w)
                if grad(x,y,action)@w>maxx:
                    maxx = grad(x,y,action)@w
                    index = action
            tmp_policy.append(index)
            
        # print(tmp_policy)
        policy = np.eye(5)[tmp_policy]  #qtable的最优值作为更新策略，并用独热码来表示
        policy_epsilon = np.vectorize(d.get)(policy)

        
    
        #输出每个state的访问次数
        # print(np.array(cnt).reshape(5,5)) 

        # state_value = np.sum(policy_epsilon * action_value,axis=1)
        # mean_state_value = np.sum(policy_epsilon * action_value,axis=1).mean()
        if episode % 10 == 0:
            print("episode",f"{episode}/{num_episodes}")
            print((TD_error * grad(2,0,tmpaction)).mean())
            gridworld.showPolicy(policy)
        # print(grid())
        # print(np.round(state_value,decimals=4).reshape(5,5))
        # print("mean_state_value", mean_state_value)



    return gridworld
    

In [94]:
x,y,z=0,0,0
len([1, x, y, z, x*x, y*y, z*z, x*y, x*z, y*z, x**3, y**3, z**3, x*x*y, x*y*y, x*x*z, x*z*z, y*y*z, y*z*z])

19

In [95]:
grad = lambda x,y,z: np.array([1, x, y, z, x*x, y*y, z*z, x*y, x*z, y*z, x**3, y**3, z**3, x*x*y, x*y*y, x*x*z, x*z*z, y*y*z, y*z*z] )
np.random.seed(1)
w = np.random.rand(19)
SARSA(gridworld, grad=grad, w=w, learning_rate=0.00001)

p1 0.6008 p0 0.0998
episode 0/600
1.88638400547647
➡️➡️⬇️⬅️⬅️
➡️⏩️⏩️⬇️⬇️
➡️➡️⏩️⬇️⬇️
➡️⏩️✅⏩️⬇️
➡️⏩️➡️➡️⬇️
episode 10/600
7.557626429406723
⬆️⬆️⬆️⬆️🔄
⬆️⏫️⏫️⬆️🔄
⬆️⬆️⏫️⬆️⬆️
⬆️⏫️✅⏫️⬆️
⬆️⏫️⬆️⬆️⬆️
episode 20/600
9.103660953030655
⬆️⬆️⬆️⬆️🔄
⬆️⏫️⏫️⬆️🔄
⬆️⬆️⏫️⬆️⬆️
⬆️⏫️✅⏫️⬆️
⬆️⏫️⬆️⬆️⬆️
episode 30/600
10.682035929716506
⬆️⬆️⬆️⬆️🔄
⬆️⏫️⏫️⬆️🔄
⬆️⬆️⏫️⬆️⬆️
⬆️⏫️✅⏫️⬆️
⬆️⏫️⬆️⬆️⬆️
episode 40/600
12.11128783835394
⬆️⬆️⬆️⬆️🔄
⬆️⏫️⏫️⬆️🔄
⬆️⬆️⏫️⬆️⬆️
⬆️⏫️✅⏫️⬆️
⬆️⏫️⬆️⬆️⬆️
episode 50/600
13.375635130923431
⬆️⬆️⬆️⬆️🔄
⬆️⏫️⏫️⬆️🔄
⬆️⬆️⏫️⬆️⬆️
⬆️⏫️✅⏫️⬆️
⬆️⏫️⬆️⬆️⬆️
episode 60/600
14.494727072851507
⬆️⬆️⬆️⬆️🔄
⬆️⏫️⏫️⬆️🔄
⬆️⬆️⏫️⬆️⬆️
⬆️⏫️✅⏫️⬆️
⬆️⏫️⬆️⬆️⬆️
episode 70/600
15.490092137655276
⬆️⬆️⬆️⬆️🔄
⬆️⏫️⏫️⬆️⬆️
⬆️⬆️⏫️⬆️⬆️
⬆️⏫️✅⏫️⬆️
⬆️⏫️⬆️⬆️⬆️
episode 80/600
16.37936752030484
⬆️⬆️⬆️⬆️🔄
⬆️⏫️⏫️⬆️⬆️
⬆️⬆️⏫️⬆️⬆️
⬆️⏫️✅⏫️⬆️
⬆️⏫️⬆️⬆️⬆️
episode 90/600
17.17641462285105
⬆️⬆️⬆️⬆️🔄
⬆️⏫️⏫️⬆️⬆️
⬆️⬆️⏫️⬆️⬆️
⬆️⏫️✅⏫️⬆️
⬆️⏫️⬆️⬆️⬆️
episode 100/600
17.89227496964756
⬆️⬆️⬆️⬆️🔄
⬆️⏫️⏫️⬆️⬆️
⬆️⬆️⏫️⬆️⬆️
⬆️⏫️✅⏫️⬆️
⬆️⏫️⬆️⬆️⬆️
episode 110/600
18.535978164995605
⬆

KeyboardInterrupt: 